<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [163]:
import math
import pandas as pd
import csv
import json
import numpy as np
from collections import defaultdict
from typing import List, Union
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/mhdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mhdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mhdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [164]:
class Preprocessor:

    def __init__(self, stopwords):
        self.stopwords = stopwords
        self.ps = PorterStemmer()
        self.lem = WordNetLemmatizer()


    def preprocess(self, text, is_query=False):
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        tokens = self.word_tokenize(text)
        tokens = self.normalize(tokens)
        if not is_query:
            tokens = self.remove_stopwords(tokens)
        return tokens

    def normalize(self, tokens):
        for i, token in enumerate(tokens):
            token = token.lower()
            token = self.ps.stem(token)
            tokens[i] = self.lem.lemmatize(token)
        return tokens
    
    def remove_links(self, text):
        text = re.sub('http[s]?://\S+', '', text)        
        return text

    def remove_punctuations(self, text):
        regular_expression = r'[^\w\s]'
        return re.sub(regular_expression, ' ', text.lower())

    def word_tokenize(self, text):
        tokenized_text = nltk.tokenize.word_tokenize(text)
        return tokenized_text

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

preprocessor = Preprocessor(stopwords=stopwords.words('english'))

In [165]:
## 1 points

data = {}
index_to_paperId = {}
paperId_to_index = {}
doc_size = {}
index = 1

with open('data/phase-2/data.csv', 'r', encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for it, row in enumerate(csv_reader):
        if it == 0:
            continue
            
        paperId = row[0]
        title = row[1]
        abstract = row[2]

        title_tokens = preprocessor.preprocess(title)
        abstract_tokens = preprocessor.preprocess(abstract)

        index_to_paperId[index] = paperId
        paperId_to_index[paperId] = index
        data[index] = (title_tokens, abstract_tokens)
        doc_size[index] = len(title_tokens) + len(abstract_tokens)

        index += 1


In [166]:
courps_size = 0
for d in doc_size:
    courps_size += doc_size[d]

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [167]:
def find_indexes(doc, word: str):
    indexes = []

    for iter, token in enumerate(doc):
        if token == word:
            indexes.append(iter + 1)

    indexes.sort()
    return indexes


def construct_positional_indexes(data: defaultdict):

    """
    Get processed data and insert words in that into a trie and construct postional_index and posting lists after wards.

    Parameters
    ----------
    corpus: str
        processed data 
    
    Return
    ----------
    docs: 
        list of docs with specificied id, title, abstract.
    """
    index = {}
    all_tokens_courps = {}
    for paper_id, (title, abstract) in data.items():
        for i, word in enumerate(title):
            if word not in index:
                index[word] = {'title': {}, 'abstract': {}}
            if word not in all_tokens_courps:
                all_tokens_courps[word] = 0
            index[word]['title'][paper_id] = find_indexes(title, word)
            all_tokens_courps[word] += len(index[word]['title'][paper_id])
            
        for i, word in enumerate(abstract):
            if word not in index:
                index[word] = {'title': {}, 'abstract': {}}
            if word not in all_tokens_courps:
                all_tokens_courps[word] = 0
            index[word]['abstract'][paper_id] = find_indexes(abstract, word)
            all_tokens_courps[word] += len(index[word]['abstract'][paper_id])

    return index, all_tokens_courps

positional_index, all_tokens_courps = construct_positional_indexes(data)

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [177]:
def get_tf_doc(token: str, positional_index: dict, doc_id: int, field) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """
    if token not in positional_index:
        return 0.1
    if doc_id in positional_index[token][field]:
        return len(positional_index[token][field][doc_id])
    return 0.1


def get_tf_query(query) -> dict:
    query_tf = {}
    for token in query:
        if token not in query_tf:
            query_tf[token] = 0
        query_tf[token] += 1
    
    return query_tf


In [178]:
def get_idf(token: str, positional_index, field) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    df = len(positional_index[token][field].keys())
    
    return np.log10(len(data.keys()) / df)


<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [209]:
def generate_tfidf_list(positional_index: dict):
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """
    tf_idf = {}
    for token in positional_index.keys():
        if token not in tf_idf:
            tf_idf[token] = {}
        for doc in positional_index[token]['abstract']:
            if doc not in tf_idf[token]:
                tf_idf[token][doc] = {'abstract': 0, 'title': 0}

            idf = get_idf(token, positional_index, 'abstract')
            tf = np.log10(get_tf_doc(token, positional_index, doc, 'abstract')) + 1

            tf_idf[token][doc]['abstract'] = tf * idf
        for doc in positional_index[token]['title']:
            if doc not in tf_idf[token]:
                tf_idf[token][doc] = {'abstract': 0, 'title': 0}

            idf = get_idf(token, positional_index, 'title')
            tf = np.log10(get_tf_doc(token, positional_index, doc, 'title')) + 1

            tf_idf[token][doc]['title'] = tf * idf
    
    return tf_idf
        

tf_idf = generate_tfidf_list(positional_index)

<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [224]:
def ltc_lnc(query_tokens, positional_index, tf_idf, how='abstract'):
    query_tf = get_tf_query(query_tokens)
    query_tf_idf = {}
    for token in query_tokens:
        query_tf_idf[token] = (np.log10(query_tf[token]) + 1)
    
    doc_scores = {}
    docs_tdidf = {}
    for token in query_tokens:
        docs = positional_index[token][how]
        for doc in docs:
            if doc not in doc_scores:
                doc_scores[doc] = 0
                docs_tdidf[doc] = []
            doc_scores[doc] += query_tf_idf[token] * tf_idf[token][doc][how]
            docs_tdidf[doc].append(tf_idf[token][doc][how])
    
    for doc in doc_scores:
        cosine = np.sqrt(np.sum(np.square(docs_tdidf[doc])))
        doc_scores[doc] /= cosine

    return doc_scores


In [225]:
query = "Translation Model Based on Deep Learning"
query_tokens = preprocessor.preprocess(query)
print(ltc_lnc(query_tokens, positional_index, tf_idf))

{74: 1.568350158178814, 100: 1.372365653919981, 205: 1.3804811873376066, 253: 1.0, 328: 1.0, 339: 1.0, 340: 1.3032964544797219, 487: 1.0, 502: 1.0, 503: 1.2902426430552718, 517: 1.0, 544: 1.0, 705: 1.3913217757648748, 739: 1.0, 750: 1.383919533901908, 753: 1.3032964544797219, 790: 1.7170725470612374, 857: 1.3474173803947083, 866: 1.619111363103459, 910: 1.0, 931: 1.372365653919981, 993: 1.0, 994: 1.0, 1076: 1.2984481150948406, 1113: 1.0, 1214: 1.5166617464731365, 1219: 1.211196504699724, 1242: 1.2984481150948406, 1278: 1.6945280397570697, 1291: 1.6213332912202298, 1356: 1.0, 1368: 1.0, 1459: 1.0, 1533: 1.4826941259904116, 1572: 1.3032964544797219, 1652: 1.8787804087179147, 1779: 1.6365038258148699, 1791: 1.367649051028664, 1802: 1.0, 1814: 1.0, 1855: 1.3421511956466716, 1914: 1.8982494221741655, 1925: 1.0, 2008: 1.732586506629078, 2015: 1.3517993471626013, 2081: 1.4259469118552173, 2140: 1.372365653919981, 2144: 1.0, 2230: 1.6066372505067232, 2274: 1.368495439913006, 2316: 1.3839195339

In [196]:
def avg_doc_len(data, field):
    """
    Computes the average document length
    """
    total_len = 0
    for docId in data.keys():
        total_len += len(data[docId][field])
    return total_len / len(data.keys())

def okapi_bm25(query_tokens, positional_index, k1=1.2, b=0.75, how='abstract'):
    doc_scores = {}
    field = 0 if how == 'title' else 1
    avg_dl = avg_doc_len(data, field)

    for token in query_tokens:
        docs = positional_index[token][how]
        idf = get_idf(token, positional_index, how)
        for doc in docs:
            if doc not in doc_scores:
                doc_scores[doc] = 0
            tf = get_tf_doc(token, positional_index, doc, how)
            doc_scores[doc] += idf * ((k1 + 1) * tf / (tf + k1 * (1 - b + b * len(data[doc][field]) / avg_dl)))

    return doc_scores

In [197]:
query = "Translation Model Based on Deep Learning"
query_tokens = preprocessor.preprocess(query)
print(okapi_bm25(query_tokens, positional_index))

{74: 0.8016047231382865, 100: 2.549338944226796, 205: 1.8271797558067926, 253: 1.4683773313959327, 328: 0.992491681847437, 339: 1.4306180105914965, 340: 1.9069340057639965, 487: 1.2257388401392462, 502: 1.0653462983970179, 503: 4.037519866878989, 517: 1.4748651927659433, 544: 1.7002340211276117, 705: 1.1955483412551637, 739: 1.107772504023275, 750: 2.3852651091228494, 753: 1.6696915015794755, 790: 3.741671748178229, 857: 1.9318097535660441, 866: 2.3033518579601604, 910: 1.4367757918019435, 931: 2.046576938025652, 993: 1.3334630125623117, 994: 2.8314136305512774, 1076: 1.798269924888257, 1113: 1.4184594497398628, 1214: 3.982839000417335, 1219: 2.9776214505039285, 1242: 1.893174347601694, 1278: 3.5843366534749217, 1291: 3.281857735859842, 1356: 2.3320643901583047, 1368: 1.2036396678251418, 1459: 1.4124573414171606, 1533: 3.047904040380687, 1572: 1.6579051748910325, 1652: 3.982927596094884, 1779: 3.8635636580766612, 1791: 4.5160283816979465, 1802: 1.107772504023275, 1814: 1.22573884013924

In [226]:
def merge_doc_scores(doc_scores1: dict, doc_scores2: dict, weight: float) -> dict:
    merged_doc_scores = {}
    for doc in doc_scores1.keys():
        merged_doc_scores[doc] = weight * doc_scores1[doc]

    for doc in doc_scores2.keys():
        if doc in merged_doc_scores:
            merged_doc_scores[doc] += (1 - weight) * doc_scores2[doc]
        else:
            merged_doc_scores[doc] = (1 - weight) * doc_scores2[doc]

    return merged_doc_scores

def search(query: str, max_result_count: int, method: str = 'ltc-lnc', weight: float = 0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_resu/lt_count = -1, then you have to return all docs

    method: 'ltc-lnc' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    query_token = preprocessor.preprocess(query)

    if method == 'ltc-lnc':
        doc_scores_abstract = ltc_lnc(query_token, positional_index, tf_idf, how='abstract')
        doc_scores_title = ltc_lnc(query_token, positional_index, tf_idf, how='title')
        doc_scores = merge_doc_scores(doc_scores_title, doc_scores_abstract, weight)
    
    elif method == 'okapi25':
        doc_scores_abstract = okapi_bm25(query_token, positional_index, how='abstract')
        doc_scores_title = okapi_bm25(query_token, positional_index, how='title')
        doc_scores = merge_doc_scores(doc_scores_title, doc_scores_abstract, weight)
        
    sorted_docs = [index_to_paperId[d[0]] for d in sorted(doc_scores.items(), key=lambda x:x[1], reverse=True)] 
    if len(sorted_docs) < max_result_count:
        return sorted_docs
    return sorted_docs[:max_result_count]

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [229]:
def create_model(positional_index):
    llm = {}
    for token in positional_index:
        if token not in llm:
            llm[token] = {}
        for doc in positional_index[token]['title']:
            if doc not in llm[token]:
                llm[token][doc] = 0
            llm[token][doc] += get_tf_doc(token, positional_index, doc, 'title')
        for doc in positional_index[token]['abstract']:
            if doc not in llm[token]:
                llm[token][doc] = 0
            llm[token][doc] += get_tf_doc(token, positional_index, doc, 'abstract')
        
    return llm


def lm_search(query: str, max_result_count: int, model: dict, l=0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    query_tokens = preprocessor.preprocess(query)

    prob = {}
    for token in query_tokens:
        for doc in model[token]:
            if doc not in prob:
                prob[doc] = 1
            if model[token][doc] == 0:
                continue
            prob[doc] *= model[token][doc]
    
    for token in query_tokens:
        for doc in prob:
            prob[doc] = prob[doc] * l + (1-l) * all_tokens_courps[token]
            
    sorted_docs = [index_to_paperId[d[0]] for d in sorted(prob.items(), key=lambda x:x[1], reverse=True)] 
    if len(sorted_docs) < max_result_count:
        return sorted_docs
    return sorted_docs[:max_result_count]
    

In [230]:
query = "Translation Model Based on Deep Learning"
llm_model = create_model(positional_index)
print(lm_search(query, 10, llm_model, l=1))


['2c88c3d8bc68858e4c9ba58ec3d71454e94eb96e', '1167a9431e3743e16ab7e38bf60bdf63a278f7e9', '0ca95a34c0a59132e7f0925b189de884e51a93a6', '9cc8609f904c50b1be408abede7ab7f5cdbc9744', 'f29df408f1407c60b0f85dd595a883de7c66e763', '924385a663f9bc3a5f6979f72b43e069e06c263c', 'b2d1008ec02fc29478737daef710ba4436b5aafa', 'a1585ce06007b95185a47929fa9e8bd0048af747', 'ddf52ceb8452344f4b93c87c0ec97834cbdd7230', '5d68bbeb2d293cbda75994ca5040fd083665e82e']


<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [199]:
class Evaluation:
    def evaluate(self, require_scores, actual, predicted):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to calculated

        Returns
        -------
        """
        for method in require_scores:
            func = getattr(Evaluation, method)
            print(f'method {method}:{func(actual, predicted)}')

    @staticmethod
    def precision(actual, predicted):
        """
        Calculates the precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The precision of the predicted results
        """
        precision = 0

        for i in range(len(predicted)):
            actual_set = actual[i]
            predicted_set = predicted[i]
            precision += len(set(actual_set) & set(predicted_set)) / len(predicted[i])
        
        return precision / len(actual)

    @staticmethod
    def recall(actual, predicted) -> float:
        """
        Calculates the recall of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The recall of the predicted results
        """
        recall = 0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            recall += len(set(actual_set) & set(predicted_set)) / len(actual[i])
        
        return recall / len(actual)

    @staticmethod
    def f1(actual, predicted) -> float:
        """
        Calculates the F1 score of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The F1 score of the predicted results    
        """
        precision = Evaluation.precision(actual, predicted)
        recall = Evaluation.recall(actual, predicted)

        return (2 * precision * recall) / (precision + recall)

    @staticmethod
    def map(actual, predicted) -> float:
        """
        Calculates the Mean Average Precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The Mean Average Precision of the predicted results
        """
        map = 0.0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            precision = 0.0
            for j in range(len(predicted_set)):
                if predicted_set[j] in actual_set:
                    precision += Evaluation.precision(actual_set, predicted_set[:j+1])
            
            map += precision / len(actual_set)

        return map

    @staticmethod
    def ndcg(actual, predicted) -> float:
        """
        Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The NDCG of the predicted results
        """
        ndcg = 0.0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            dcg = 0.0
            idcg = 0.0
            for j in range(len(predicted_set)):
                if predicted_set[j] in actual_set:
                    dcg += 1 / np.log2(j + 2)
                idcg += 1 / np.log2(j + 2)
            
            ndcg += dcg / idcg

        return ndcg / len(actual)
    
    @staticmethod
    def mrr(actual, predicted) -> float:
        """
        Calculates the Mean Reciprocal Rank of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The MRR of the predicted results
        """
        MRR = 0.0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            for j in range(len(predicted_set)):
                if predicted_set[j] in actual_set:
                    MRR += 1 / (j + 1)
                    break

        return MRR / len(actual)

In [231]:
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

with open('./data/phase-2/validation.json', 'r') as f:
    validation = json.load(f)

queries = list(validation.keys())
actuals = [validation[query] for query in queries]

In [232]:
predicts = [search(query, 10, method='ltc-lnc', weight=0.8) for query in queries]
print(predicts)

Evaluation().evaluate(require_scores, actuals, predicts)

[['0ca95a34c0a59132e7f0925b189de884e51a93a6', '864e033c002c5ec48d4c273c53ce995682fc3e21', '1167a9431e3743e16ab7e38bf60bdf63a278f7e9', '9cc8609f904c50b1be408abede7ab7f5cdbc9744', '924385a663f9bc3a5f6979f72b43e069e06c263c', '20f77d34ab1aec7d0a6e613a740aff7ec7fbf55a', '2c88c3d8bc68858e4c9ba58ec3d71454e94eb96e', 'f29df408f1407c60b0f85dd595a883de7c66e763', '5c6199859eb8b4ebd4188c6587f6ad8e6448a983', 'b3ea79473e7438ae3d551f79a84207f8bd0e830d'], ['081651b38ff7533550a3adfc1c00da333a8fe86c', '6ce6768a34e197e4ef76933c8e6760b6e4833dc4', 'e64f6ef3e16ed0ce08710fb9eb7198dc021ec8cf', 'daf74c34f7da0695b154f645c8b78a7397a98f16', 'afff02fa27bdeb0e2dc658d1ce43af6d3f407ff5', '2e2b189f668cf2c06ebc44dc9b166648256cf457', '05fd1da7b2e34f86ec7f010bef068717ae964332', '6bd36e9fd0ef20a3074e1430a6cc601e6d407fc3', '82a99e0d626847fadb6be938f979a4aec573e9a1', 'da26630c9d931952f1d68308148f1289f5332607'], ['acdf02ca0bee200748f7007c8b0f78a8e79b68e6', '7bfd24b2e5029b53a2289a5ff52cf9b2ebb187c5', 'a4a047f23b29ac3e88f3f8b53

In [233]:
predicts = [search(query, 10, method='okapi25', weight=0.8) for query in queries]
print(predicts)

Evaluation().evaluate(require_scores, actuals, predicts)

[['1167a9431e3743e16ab7e38bf60bdf63a278f7e9', '864e033c002c5ec48d4c273c53ce995682fc3e21', '0ca95a34c0a59132e7f0925b189de884e51a93a6', '9cc8609f904c50b1be408abede7ab7f5cdbc9744', 'b3ea79473e7438ae3d551f79a84207f8bd0e830d', 'f29df408f1407c60b0f85dd595a883de7c66e763', '2c88c3d8bc68858e4c9ba58ec3d71454e94eb96e', '20f77d34ab1aec7d0a6e613a740aff7ec7fbf55a', '3060a4828f4fd0a89e934db5b74bfdef95141d80', '4dfbc372c5e127abe7989e9c4ef75daac7370cd2'], ['081651b38ff7533550a3adfc1c00da333a8fe86c', '05fd1da7b2e34f86ec7f010bef068717ae964332', '25e3cad9f9f7191e5057975c9c2d3f2fda24ff96', '3dda2a28f8b7ddb89bfaa3bf1f1577d30afc7f2a', '6ce6768a34e197e4ef76933c8e6760b6e4833dc4', 'd6fff78f45db5e8a6246d256d58a047c7647059f', 'daf74c34f7da0695b154f645c8b78a7397a98f16', '642d0f49b7826adcf986616f4af77e736229990f', '3e6c84302b2b56cf8369253d6168b852d0aa1fd6', '791b65c65f8ae7e16c1ee9203cdc3ee59ffeb99f'], ['acdf02ca0bee200748f7007c8b0f78a8e79b68e6', 'a4a047f23b29ac3e88f3f8b53e8f4ad8170ad38f', '7bfd24b2e5029b53a2289a5ff

In [238]:
llm_model = create_model(positional_index)
predicts = [lm_search(query, 10, llm_model, 1) for query in queries]
print(predicts)

Evaluation().evaluate(require_scores, actuals, predicts)

[['2c88c3d8bc68858e4c9ba58ec3d71454e94eb96e', '1167a9431e3743e16ab7e38bf60bdf63a278f7e9', '0ca95a34c0a59132e7f0925b189de884e51a93a6', '9cc8609f904c50b1be408abede7ab7f5cdbc9744', 'f29df408f1407c60b0f85dd595a883de7c66e763', '924385a663f9bc3a5f6979f72b43e069e06c263c', 'b2d1008ec02fc29478737daef710ba4436b5aafa', 'a1585ce06007b95185a47929fa9e8bd0048af747', 'ddf52ceb8452344f4b93c87c0ec97834cbdd7230', '5d68bbeb2d293cbda75994ca5040fd083665e82e'], ['5ff28f5044211577799d1383fbcc5f3e93ea56a3', 'afff02fa27bdeb0e2dc658d1ce43af6d3f407ff5', '1a3c74c7b11ad5635570932577cdde2a3f7a6a5c', 'ddf52ceb8452344f4b93c87c0ec97834cbdd7230', '05fd1da7b2e34f86ec7f010bef068717ae964332', '642d0f49b7826adcf986616f4af77e736229990f', '081651b38ff7533550a3adfc1c00da333a8fe86c', '3e6c84302b2b56cf8369253d6168b852d0aa1fd6', '3a7f661d157cfb689bb35969e1a0fccccf8ba698', '20f77d34ab1aec7d0a6e613a740aff7ec7fbf55a'], ['acdf02ca0bee200748f7007c8b0f78a8e79b68e6', '3f503da9263298b2831e702dc4daad355da1f5e8', '7bfd24b2e5029b53a2289a5ff